# Selecteren parameters DBScan
Met dit script testen we welke parameters alleen de stoplocaties clusteren met als doel dat de computer straks zelf de juiste parameters kan selecteren.  

Om dit te testen, bouwen we een for loop die meerdere malen de DBScan uitvoert met verschillende combinaties van de parameters epsilon en minimal samples. 

Om te zicht te krijgen in welke combinatie van epsilon en minimal samples leidt tot groot verschil in distributie tussen clusters en ruis, geven we elke combinatie van parameters een u-score (Mann-Withney U). Waarbij een lage u-score staat voor veel verschil in distributie en een hoge u-score staat voor minder verschil in distributie. Maar let op: de parameters met de laagste u-score betekend niet dat er alleen stoplocaties zijn geclusterd.  

In [ ]:
# Laat packages
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn import preprocessing
import seaborn as sns
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
import sklearn
import multiprocessing as mp

In [ ]:
# Laat de pre-processed data
df = pd.read_csv("sample_AIS_preprocessing.csv")

##### Selecteer features DBScan
Features zijn de waarden die wij meegeven als input aan de DBScan. Wij geven latitude/ longitude mee ter bepaling van de locatie en die punten te clusteren die dicht bij elkaar liggen. Snelheid geven mee om die punten te clusteren die vergelijkbare snelheid hebben of in het geval van stoplocaties een lage snelheid hebben. 

Naast de bovenstaande features willen verschillende features testen. Namelijk volgnummer vs. volgnummer met tijdselement en koersstabiliteit. Het meenemen van volgnummer zorgt ervoor dat zichtbaar wordt hoevaak iemand op dezelfde locatie geweest is. Het geeft een rangorde aan de punten, waardoor er onderscheidt gemaakt wordt tussen punten die dichtbij elkaar liggen qua afstand en snelheid, maar wel verschillen in tijd. Een afweging is of je alleen rangorde (bijv. 1,2,3,4,5) wil meegeven, of dat je daaraan nog een extra tijdselement (bijv. 1 ,40, 60, 180) aan toe wil voegen. In dit geval, berekenen je het verschil in seconden tussen twee punten. Daarnaast kun je koersstabiliteit meegeven. Naar verwachting is dat de koersstabiliteit verschilt tussen clusters en ruis. 

In [ ]:
#Features worden aan de DBScan gegeven in vorm van een array. De waarden worden genormaliseerd door de functie scale. De waarden variëeren nu tussen -5 en 5. 
## Features die je kunt meenemen zijn: t_latitude (latitude), t_longitude (longitude), t_speed (speed), VgNr (volgnummer met tijdselement), ID (volgnummer zonder tijdselement), angle_deg_diff(koersstabiliteit: eerste afgeleide van hoek tussen twee punten in graden 0-180)
coords_speed = np.asarray(df[["t_latitude", "t_longitude", "t_speed", "VgNr"]])
coords = preprocessing.scale(coords_speed)

In [ ]:
# Creëer functie die per parameter combinatie aangeeft wat de u-waarden zijn.
def gemiddelde_kmu_per_eps(df, coords, eps, min_samples, speed_column="t_speed"):
    # Hier run je de DBScan
    model = DBSCAN(eps=eps, min_samples=min_samples, algorithm="auto").fit(coords)
    # Cluster labels opvragen
    cluster_labels = model.labels_
    # Aantal clusters
    num_clusters = len(set(cluster_labels))
    # clusters labels naar kolom
    df["cluster"] = cluster_labels
    
    # selecteer snelheidskolom van clusters en berekenen gemiddelde snelheid en aantal punten in cluster 
    clusters = df.t_speed[df["cluster"] > -1]
    # Berekenen gemiddelde snelheid en tel het aantal punten in de cluster kolom
    mean_kmu_cluster = clusters.mean()
    count_cluster = clusters.count()
    
    #  selecteer snelheidskolom van ruis en berekenen gemiddelde snelheid en aantal punten 
    ruis = df.t_speed[df["cluster"] == -1]
    # Berekenen gemiddelde snelheid en tel aantal punten in de ruis kolom
    mean_kmu_ruis = ruis.mean()
    count_ruis = ruis.count()

    # Verschil snelheidskolom van cluster vs ruis in u waarden
    u, prob = mannwhitneyu(clusters, ruis)
    
    return(eps, min_samples, num_clusters, count_cluster, mean_kmu_cluster, count_ruis, mean_kmu_ruis, u, prob)

# Bovenstaande functie uitvoeren voor 100 verschillende epsilons en 45 verschillende MinPoints combinaties
output_list = []
for epsilon in np.linspace(1e-4, 1, num=100):
    for min_samples in range(2, 45, 1):
        # Hier roep je functie aan
        output = gemiddelde_kmu_per_eps(df=df, coords=coords, eps=epsilon, min_samples=min_samples)
        output_list.append(output)

# Output functie naar dataframe en csv (nu hoef je functie maar 1x te runnen)
output_df = pd.DataFrame(output_list, columns = ["eps", "min_samples", "N_cluster", "count_cluster", "mean_kmu_cluster", "count_ruis", "mean_kmu_ruis", "u", "prob"])
output_df.to_csv("sample_output.csv")